### **Function Definitions**

#### *Given requirements (for the data retrieved):*
- It is preferred to have the **gene name** in title and/or abstract. Use search tag *[tiab]* for this in the PubMed database. PMC database simply ignores [tiab] as it is an unknown field in that db. 
- The publication must be well-cited by having at least **25** articles that cited it.
- For a query that includes "Ivermectin", exclude articles relating to COVID-19 with *"NOT covid-19"* in the query.

Return the article IDs for the results that meet the criteria above.

In [1]:
import time
from Bio import Entrez
import pandas as pd
import numpy as np


## Function to get query counts for PubMed and PMC databases only
def global_db_search(query_list, genes_list=[]):

    if len(genes_list) != 0:
        for i in query_list:
            for j in genes_list:
                query = i + j + "[tiab]"

                handle = Entrez.egquery(term=query)
                record = Entrez.read(handle)
                df = pd.DataFrame(record["eGQueryResult"]).head(2)
                df["Query"] = query
                append_data(df, 'global_query_res.csv', False)
                time.sleep(0.34)
    else:
        for i in query_list:
            handle = Entrez.egquery(term=i)
            record = Entrez.read(handle)
            df = pd.DataFrame(record["eGQueryResult"]).head(2)
            df["Query"] = i
            append_data(df, 'global_query_res.csv', False)
            time.sleep(0.34)
    return


## Function for reading in the df "summary" results
def read_in_results(file_name):

    # The converters are there so that each list is NOT inside a string
    res_df = pd.read_csv(file_name,  converters={"MainID_List": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "P_Dates": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "P_Years": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "LinkedID_List": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "Query_Count": int})
    return res_df


def esummary_info(in_webenv_key, in_query_key, db_name):

    # Obtaining DocSums for a set of IDs that are stored on the Entrez History server.
    handle = Entrez.esummary(db=db_name, webenv=in_webenv_key, query_key=in_query_key)
    record = Entrez.read(handle)

    publ_dates, publ_years = get_published_dates(record)

    if db_name == "pubmed":
        ids_list = get_pmcids(record)
    else:
        ids_list = get_pmids(record)

    return publ_dates, publ_years, ids_list


def get_published_dates(esummary_rec):

    retr_dates = []
    retr_years = []

    for article in esummary_rec:
        # "PubDate" is often of the form: '2021 Nov 26'
        retr_dates.append(article["PubDate"])
        p_year = int(article["PubDate"].split()[0])
        retr_years.append(p_year)

    return retr_dates, retr_years


def get_pmcids(esummary_rec):

    pmcids_list = []
    for i in esummary_rec:
        # If "pmc" is there, then this article also has a PMCID (i.e., it's also found in the PubMed Central db)
        if "pmc" in i["ArticleIds"]:
            pmcids_list.append(i["ArticleIds"]["pmc"])
        else:
            pmcids_list.append(np.NaN)

    return pmcids_list


def get_pmids(esummary_rec):

    pmids_list = []
    for i in esummary_rec:
        # '0' means that the article has no PMID (i.e., it's not found in the PubMed db)
        if i["ArticleIds"]["pmid"] == '0':
            pmids_list.append(np.NaN)
        else:
            pmids_list.append(i["ArticleIds"]["pmid"])

    return pmids_list


## Function that retrieves summary results from a given set of queries (which don't require a gene list)
def get_query_info_no_genes(query_in, db_name):

    # relevance: Records are sorted based on relevance to your search. (Relevance ranking)
    search_results = Entrez.read(
        Entrez.esearch(db=db_name, term=query_in, sort="relevance", retmax=5000, usehistory="y")
        )

    # NEED TO FIRST CHECK IF WE GOT ANY RESULTS FROM THAT QUERY
    if len(search_results["IdList"]) == 0:
        return
    else:
        # With search_results, we will use its WebEnv value and QueryKey value
        p_dates, p_years, ids_list = esummary_info(search_results["WebEnv"], search_results["QueryKey"], db_name)

        time.sleep(0.34)

        return pd.DataFrame([[query_in, db_name, search_results['Count'], search_results['IdList'], p_dates, p_years, ids_list]],
                                columns=['Query', 'Db_Name', 'Query_Count', 'MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'])


## Function that retrieves summary results from a given set of queries (which requires a gene list)
def get_query_info(query_in, genes, db_name):

    gene_query = []
    query = ""

    for i in genes:
        # Example of db_name values in this use case: "pubmed" or "pmc"
        if db_name == "pubmed":
            # PubMed's Search field tag: Title/Abstract [tiab]
            query = query_in + i + "[tiab]"
        else:
            query = query_in + i

        # relevance: Records are sorted based on relevance to your search. (Relevance ranking)
        search_results = Entrez.read(
            Entrez.esearch(db=db_name, term=query, sort="relevance", retmax=5000, usehistory="y")
            )
     
        # NEED TO FIRST CHECK IF WE GOT ANY RESULTS FROM THAT QUERY
        if len(search_results["IdList"]) == 0:
            continue

        # With search_results, we will use its WebEnv value and QueryKey value
        p_dates, p_years, ids_list = esummary_info(search_results["WebEnv"], search_results["QueryKey"], db_name)
       
        gene_query.append([query, db_name, search_results['Count'], search_results['IdList'], p_dates, p_years, ids_list])
        time.sleep(0.34)
          
    return pd.DataFrame(gene_query, columns=['Query', 'Db_Name', 'Query_Count', 'MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'])


## Function for obtaining citation counts for the set of IDs found in the "summary" df
def cited_cnt_table(df_summary, db_name):

    elink_data = []
    link_name = ""

    if db_name == "pubmed":
        link_name = "pubmed_pubmed_citedin"
    else:
        link_name = "pmc_pmc_citedby"  # "pmc" is the other db_name in this use case

    for i in range(0, len(df_summary)):

        query_term = df_summary.iloc[i]["Query"]

        for id_num in df_summary.iloc[i]["MainID_List"]:

            record = Entrez.read(Entrez.elink(id=id_num, dbfrom=db_name, db=db_name, linkname=link_name))
         
            if len(record[0]["LinkSetDb"]) != 0:
                cited_counts = len(record[0]["LinkSetDb"][0]["Link"])
            else:
                # 'LinkSetDb' key contains empty list when an article has no citation counts
                cited_counts = 0
            elink_data.append([query_term, db_name, id_num, cited_counts])

            if (df_summary.iloc[i]["MainID_List"].index(id_num) + 1) % 3 == 0:
                time.sleep(0.34)

    return pd.DataFrame(elink_data, columns=["Query", "Db_Name", "Id_List", "Citation_Cnts"]) 


## Function that returns the Top-k results (pass in k as an argument to the function, input by the user)
def get_top_k(df, k_val):

    q_top_k = []

    for q in df["Query"].unique():
        matches_ids = []  # For each query version, these are the IDs meeting the criteria of having citation counts >= 25
        counts = []
        df_temp = df[df["Query"] == q]

        for i in range(0, len(df_temp)):
            if df_temp.iloc[i]["Citation_Cnts"] >= 25:
                matches_ids.append(int(df_temp.iloc[i]["Id_List"]))
                counts.append(df_temp.iloc[i]["Citation_Cnts"])
                if len(matches_ids) == k_val:
                    break
        if len(matches_ids) == 0:
            continue
        q_top_k.append([q, matches_ids, counts])

    top_k_df = pd.DataFrame(q_top_k, columns=["Query", "Top_"+str(k_val)+"_Ids", "Citation_Cnts"])

    return top_k_df


## Function that appends DataFrame rows to a CSV file
def append_data(df, file_name, is_new_file):

    if is_new_file:
        # if True, then
        df.to_csv(file_name, index=False)
    else:
        # False: This is an existing CSV file
        df.to_csv(file_name, mode='a', index=False, header=False)

    return


---

In [2]:
Entrez.email = "Your.Email.Here@example.org"  # Always tell NCBI who you are

In [3]:
genes = ['GABRA1', 'GABRA2', 'GABRA3', 'GABRA4', 'GABRA5']

query_vers_with_genes = ["GABA AND ", "Zebrafish AND ", "GABA AND Ivermectin NOT covid-19 AND ", "GABA AND Ivermectin NOT covid-19 AND Zebrafish AND "]
query_vers_no_genes = ["GABA AND Ivermectin NOT covid-19", "GABA AND Ivermectin NOT covid-19 AND Zebrafish"]

##### PubMed information retrieval

In [25]:
df_q = get_query_info(query_vers_with_genes[0], genes, "pubmed")

df_q

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1[tiab],pubmed,169,"[33825593, 30324621, 32047208, 28535436, 24613...","[2021 Mar, 2018 Nov, 2020 Feb 11, 2017 Jul, 20...","[2021, 2018, 2020, 2017, 2014, 2014, 2015, 202...","[nan, nan, PMC7012862, nan, PMC6294571, PMC392..."
1,GABA AND GABRA2[tiab],pubmed,173,"[32565954, 24613745, 18005236, 24136292, 34174...","[2020 Jul, 2014 May, 2008 Jun, 2014 Mar, 2021 ...","[2020, 2014, 2008, 2014, 2021, 2008, 2018, 201...","[PMC7286117, PMC6294571, nan, PMC3924525, PMC8..."
2,GABA AND GABRA3[tiab],pubmed,71,"[32565954, 19084931, 24040174, 19087248, 34174...","[2020 Jul, 2008 Dec 21, 2013, 2008 Dec 16, 202...","[2020, 2008, 2013, 2008, 2021, 2008, 2008, 201...","[PMC7286117, PMC2776875, PMC3764027, PMC261575..."
3,GABA AND GABRA4[tiab],pubmed,56,"[29299688, 29720720, 29445327, 26405827, 29151...","[2018 Apr, 2018 Jul, 2018, 2016 Apr, 2018 Jan,...","[2018, 2018, 2018, 2016, 2018, 2011, 2008, 200...","[nan, nan, PMC5797743, PMC4821055, PMC5792317,..."
4,GABA AND GABRA5[tiab],pubmed,105,"[33442857, 30815456, 21070817, 22383672, 12661...","[2021 Nov, 2019 Feb, 2011 Mar 1, 2012 Apr, 200...","[2021, 2019, 2011, 2012, 2003, 2021, 2009, 201...","[nan, PMC6388437, nan, nan, nan, PMC8278801, n..."


In [26]:
append_data(df_q, "pubmed_raw_res.csv", True)

In [9]:
for i in range(1, len(query_vers_with_genes)):
    df_q = get_query_info(query_vers_with_genes[i], genes, "pubmed")
    append_data(df_q, "pubmed_raw_res.csv", False)

In [10]:
for i in range(0, len(query_vers_no_genes)):
    df_q = get_query_info_no_genes(query_vers_no_genes[i], "pubmed")
    append_data(df_q, "pubmed_raw_res.csv", False)

##### PubMed Central (PMC) information retrieval

In [11]:
df_q = get_query_info(query_vers_with_genes[0], genes, "pmc")
append_data(df_q, "pmc_raw_res.csv", True)

In [12]:
for i in range(1, len(query_vers_with_genes)):
    df_q = get_query_info(query_vers_with_genes[i], genes, "pmc")
    append_data(df_q, "pmc_raw_res.csv", False)

In [13]:
for i in range(0, len(query_vers_no_genes)):
    df_q = get_query_info_no_genes(query_vers_no_genes[i], "pmc")
    append_data(df_q, "pmc_raw_res.csv", False)

---
#### Now, getting the table of citation counts for all the IDs and then obtaining the top-3 per query with the functions ***cited_cnt_table(df_summary, db_name)*** and ***get_top_k(df, k_val)***, respectively.

In [21]:
# Our PubMed summary results
df_summary = read_in_results("pubmed_raw_res.csv")

In [22]:
df_summary

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1[tiab],pubmed,169,"['33825593', '30324621', '32047208', '28535436...","['2021 Mar', '2018 Nov', '2020 Feb 11', '2017 ...","[2021, 2018, 2020, 2017, 2014, 2014, 2015, 202...","[nan, nan, 'PMC7012862', nan, 'PMC6294571', 'P..."
1,GABA AND GABRA2[tiab],pubmed,173,"['32565954', '24613745', '18005236', '24136292...","['2020 Jul', '2014 May', '2008 Jun', '2014 Mar...","[2020, 2014, 2008, 2014, 2021, 2008, 2018, 201...","['PMC7286117', 'PMC6294571', nan, 'PMC3924525'..."
2,GABA AND GABRA3[tiab],pubmed,71,"['32565954', '19084931', '24040174', '19087248...","['2020 Jul', '2008 Dec 21', '2013', '2008 Dec ...","[2020, 2008, 2013, 2008, 2021, 2008, 2008, 201...","['PMC7286117', 'PMC2776875', 'PMC3764027', 'PM..."
3,GABA AND GABRA4[tiab],pubmed,56,"['29299688', '29720720', '29445327', '26405827...","['2018 Apr', '2018 Jul', '2018', '2016 Apr', '...","[2018, 2018, 2018, 2016, 2018, 2011, 2008, 200...","[nan, nan, 'PMC5797743', 'PMC4821055', 'PMC579..."
4,GABA AND GABRA5[tiab],pubmed,105,"['33442857', '30815456', '21070817', '22383672...","['2021 Nov', '2019 Feb', '2011 Mar 1', '2012 A...","[2021, 2019, 2011, 2012, 2003, 2021, 2009, 201...","[nan, 'PMC6388437', nan, nan, nan, 'PMC8278801..."
5,Zebrafish AND GABRA1[tiab],pubmed,8,"['28535436', '32205311', '34411917', '32753576...","['2017 Jul', '2020 Apr 13', '2021 Oct', '2020 ...","[2017, 2020, 2021, 2020, 2021, 2019, 2018, 2020]","[nan, 'PMC7197724', nan, 'PMC7403336', 'PMC867..."
6,Zebrafish AND GABRA2[tiab],pubmed,3,"['32753576', '32725455', '29124181']","['2020 Aug 4', '2021 Oct', '2015 Sep']","[2020, 2021, 2015]","['PMC7403336', nan, 'PMC5668850']"
7,GABA AND Ivermectin NOT covid-19,pubmed,200,"['27543424', '27742867', '33069391', '29055807...","['2016 Oct', '2017 Jan', '2021 Mar', '2017 Dec...","[2016, 2017, 2021, 2017, 2017, 2021, 2010, 202...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
8,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pubmed,3,"['25733401', '28479061', '24040509']","['2015 May-Jun', '2017 Jun 6', '2013 Sep 10']","[2015, 2017, 2013]","[nan, nan, 'PMC3771564']"


In [23]:
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Query          9 non-null      object
 1   Db_Name        9 non-null      object
 2   Query_Count    9 non-null      object
 3   MainID_List    9 non-null      object
 4   P_Dates        9 non-null      object
 5   P_Years        9 non-null      object
 6   LinkedID_List  9 non-null      object
dtypes: object(7)
memory usage: 632.0+ bytes


In [ ]:
df_summary = read_in_results("pubmed_raw_res.csv")
df_c = cited_cnt_table(df_summary, db_name="pubmed")  # obtaining the citation counts for the PMIDs in the df_summary
append_data(df_c, "pubmed_citation_cnts.csv", True)

In [ ]:
# get_top_k() function with k_val=3
df_citations = pd.read_csv("pubmed_citation_cnts.csv")
df_top_3 = get_top_k(df_citations, 3)

In [ ]:
df_top_3

,Query,Top_3_Ids,Citation_Cnts
0,GABA AND GABRA1[tiab],"[24136292, 24361861, 20438718]","[47, 65, 40]"
1,GABA AND GABRA2[tiab],"[18005236, 24136292, 18482426]","[34, 47, 27]"
2,GABA AND GABRA3[tiab],"[18334916, 18821008, 23169495]","[34, 199, 25]"
3,GABA AND GABRA4[tiab],"[18334916, 18482426, 16901909]","[34, 27, 67]"
4,GABA AND GABRA5[tiab],"[22383672, 19809285, 29961870]","[26, 31, 25]"
5,GABA AND Ivermectin NOT covid-19,"[12421359, 23133688, 7957605]","[27, 34, 33]"


In [ ]:
append_data(df_top_3, "pubmed_top3_25cited.csv", True)

#### Now, for the **PMC** database results, before getting the top-3, we want to remove any duplicate IDs that were already retrieved from the PubMed database.

In [52]:
pmc_df = read_in_results("pmc_raw_res.csv")

In [78]:
pmc_df.columns

Index(['Query', 'Db_Name', 'Query_Count', 'MainID_List', 'P_Dates', 'P_Years',
       'LinkedID_List'],
      dtype='object')

In [80]:
# This LinkedID_List contains the corresponding PMIDs (if exists) for the publications in PMC db
pmc_df.apply(lambda x: x.explode() if x.name in ['MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'] else x)

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1,pmc,1214,6214766,2018 Sep 7,2018,30103280
0,GABA AND GABRA1,pmc,1214,6543741,2019 Jun,2019,30571139
0,GABA AND GABRA1,pmc,1214,7012862,2020 Feb 11,2020,32047208
0,GABA AND GABRA1,pmc,1214,7197724,2020 Apr 28,2020,32205311
0,GABA AND GABRA1,pmc,1214,7697095,2020 Nov 11,2020,33187258
...,...,...,...,...,...,...,...
16,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,3486162,2012 Feb 24,2012,nan
16,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8953458,2022 Mar 20,2022,35335830
16,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8233170,2021 Jun 17,2021,34195015
16,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,6174339,2018 Aug 28,2018,30344292


In [81]:
# Saving the resulting DataFrame to a variable
fin_pmc_results = pmc_df.apply(lambda x: x.explode() if x.name in ['MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'] else x)

In [87]:
append_data(fin_pmc_results, "pmc_explode_res.csv", True)

In [88]:
pub_df = read_in_results("pubmed_raw_res.csv")

# This LinkedID_List contains the corresponding PMCIDs (if exists) for the publications in PubMed db
fin_pub_results = pub_df.apply(lambda x: x.explode() if x.name in ['MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'] else x)
fin_pub_results

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1[tiab],pubmed,169,33825593,2021 Mar,2021,nan
0,GABA AND GABRA1[tiab],pubmed,169,30324621,2018 Nov,2018,nan
0,GABA AND GABRA1[tiab],pubmed,169,32047208,2020 Feb 11,2020,PMC7012862
0,GABA AND GABRA1[tiab],pubmed,169,28535436,2017 Jul,2017,nan
0,GABA AND GABRA1[tiab],pubmed,169,24613745,2014 May,2014,PMC6294571
...,...,...,...,...,...,...,...
7,GABA AND Ivermectin NOT covid-19,pubmed,200,13678839,2003 Sep,2003,nan
7,GABA AND Ivermectin NOT covid-19,pubmed,200,11714703,2002 Jan 18,2002,nan
8,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pubmed,3,25733401,2015 May-Jun,2015,nan
8,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pubmed,3,28479061,2017 Jun 6,2017,nan


In [89]:
append_data(fin_pub_results, "pubmed_explode_res.csv", True)

#### We exploded both DataFrames in order to pivot-longer, and then (via left join) we will only grab the PMCIDs *not* found in the (previously obtained) PubMed search results for the given queries.
----

In [91]:
pmc_explode = pd.read_csv("pmc_explode_res.csv", converters={"LinkedID_List": str})
pmc_explode

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1,pmc,1214,6214766,2018 Sep 7,2018,30103280
1,GABA AND GABRA1,pmc,1214,6543741,2019 Jun,2019,30571139
2,GABA AND GABRA1,pmc,1214,7012862,2020 Feb 11,2020,32047208
3,GABA AND GABRA1,pmc,1214,7197724,2020 Apr 28,2020,32205311
4,GABA AND GABRA1,pmc,1214,7697095,2020 Nov 11,2020,33187258
...,...,...,...,...,...,...,...
5446,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,3486162,2012 Feb 24,2012,nan
5447,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8953458,2022 Mar 20,2022,35335830
5448,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8233170,2021 Jun 17,2021,34195015
5449,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,6174339,2018 Aug 28,2018,30344292


In [148]:
fin_pub_results

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1[tiab],pubmed,169,33825593,2021 Mar,2021,nan
0,GABA AND GABRA1[tiab],pubmed,169,30324621,2018 Nov,2018,nan
0,GABA AND GABRA1[tiab],pubmed,169,32047208,2020 Feb 11,2020,PMC7012862
0,GABA AND GABRA1[tiab],pubmed,169,28535436,2017 Jul,2017,nan
0,GABA AND GABRA1[tiab],pubmed,169,24613745,2014 May,2014,PMC6294571
...,...,...,...,...,...,...,...
7,GABA AND Ivermectin NOT covid-19,pubmed,200,13678839,2003 Sep,2003,nan
7,GABA AND Ivermectin NOT covid-19,pubmed,200,11714703,2002 Jan 18,2002,nan
8,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pubmed,3,25733401,2015 May-Jun,2015,nan
8,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pubmed,3,28479061,2017 Jun 6,2017,nan


In [149]:
# Left join for the two DataFrames, joining on the PMIDs
pmc_explode.merge(fin_pub_results, how="left", left_on="LinkedID_List", right_on="MainID_List", suffixes=["_pmc", "_pub"], indicator=True)

,Query_pmc,Db_Name_pmc,Query_Count_pmc,MainID_List_pmc,P_Dates_pmc,P_Years_pmc,LinkedID_List_pmc,Query_pub,Db_Name_pub,Query_Count_pub,MainID_List_pub,P_Dates_pub,P_Years_pub,LinkedID_List_pub,_merge
0,GABA AND GABRA1,pmc,1214,6214766,2018 Sep 7,2018,30103280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,GABA AND GABRA1,pmc,1214,6543741,2019 Jun,2019,30571139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,GABA AND GABRA1,pmc,1214,7012862,2020 Feb 11,2020,32047208,GABA AND GABRA1[tiab],pubmed,169.0,32047208,2020 Feb 11,2020,PMC7012862,both
3,GABA AND GABRA1,pmc,1214,7197724,2020 Apr 28,2020,32205311,GABA AND GABRA1[tiab],pubmed,169.0,32205311,2020 Apr 13,2020,PMC7197724,both
4,GABA AND GABRA1,pmc,1214,7197724,2020 Apr 28,2020,32205311,Zebrafish AND GABRA1[tiab],pubmed,8.0,32205311,2020 Apr 13,2020,PMC7197724,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,3486162,2012 Feb 24,2012,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5678,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8953458,2022 Mar 20,2022,35335830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5679,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8233170,2021 Jun 17,2021,34195015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5680,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,6174339,2018 Aug 28,2018,30344292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [151]:
df_pmc = pmc_explode.merge(fin_pub_results, how="left", left_on="LinkedID_List", right_on="MainID_List", suffixes=["_pmc", "_pub"], indicator=True)

In [157]:
append_data(df_pmc, "left_join_pmc_OrderPreserved.csv", True)

By grabbing the non-matching rows from the join, we are excluding any articles previously pulled from the PubMed results, so that we're not looking at duplicate articles after querying against both databases (with PubMed being the first database, and then PMC).

In [159]:
# "left_only" signifies the NON-matching rows from the left join
fin_pmc = df_pmc[df_pmc["_merge"] == "left_only"][['Query_pmc', 'Db_Name_pmc', 'Query_Count_pmc', 'MainID_List_pmc',
                                                   'P_Dates_pmc', 'P_Years_pmc', 'LinkedID_List_pmc']]

In [160]:
fin_pmc

,Query_pmc,Db_Name_pmc,Query_Count_pmc,MainID_List_pmc,P_Dates_pmc,P_Years_pmc,LinkedID_List_pmc
0,GABA AND GABRA1,pmc,1214,6214766,2018 Sep 7,2018,30103280
1,GABA AND GABRA1,pmc,1214,6543741,2019 Jun,2019,30571139
5,GABA AND GABRA1,pmc,1214,7697095,2020 Nov 11,2020,33187258
7,GABA AND GABRA1,pmc,1214,6463728,2019 Feb 6,2019,30728247
8,GABA AND GABRA1,pmc,1214,6180030,2018 Oct 10,2018,30305619
...,...,...,...,...,...,...,...
5677,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,3486162,2012 Feb 24,2012,nan
5678,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8953458,2022 Mar 20,2022,35335830
5679,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,8233170,2021 Jun 17,2021,34195015
5680,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,6174339,2018 Aug 28,2018,30344292


In [173]:
fin_pmc["Query_pmc"].value_counts()

GABA AND GABRA1                                   1105
GABA AND Ivermectin NOT covid-19                   953
GABA AND GABRA2                                    920
GABA AND GABRA5                                    651
GABA AND GABRA4                                    424
GABA AND GABRA3                                    415
Zebrafish AND GABRA1                               133
GABA AND Ivermectin NOT covid-19 AND Zebrafish     110
Zebrafish AND GABRA2                                70
Zebrafish AND GABRA3                                52
Zebrafish AND GABRA5                                51
Zebrafish AND GABRA4                                40
GABA AND Ivermectin NOT covid-19 AND GABRA1         10
GABA AND Ivermectin NOT covid-19 AND GABRA2          7
GABA AND Ivermectin NOT covid-19 AND GABRA4          5
GABA AND Ivermectin NOT covid-19 AND GABRA5          4
GABA AND Ivermectin NOT covid-19 AND GABRA3          4
Name: Query_pmc, dtype: int64

In [162]:
# Dropping this column as it's not important for our current task
fin_pmc = fin_pmc.drop(columns="Query_Count_pmc")

In [163]:
fin_pmc

,Query_pmc,Db_Name_pmc,MainID_List_pmc,P_Dates_pmc,P_Years_pmc,LinkedID_List_pmc
0,GABA AND GABRA1,pmc,6214766,2018 Sep 7,2018,30103280
1,GABA AND GABRA1,pmc,6543741,2019 Jun,2019,30571139
5,GABA AND GABRA1,pmc,7697095,2020 Nov 11,2020,33187258
7,GABA AND GABRA1,pmc,6463728,2019 Feb 6,2019,30728247
8,GABA AND GABRA1,pmc,6180030,2018 Oct 10,2018,30305619
...,...,...,...,...,...,...
5677,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,3486162,2012 Feb 24,2012,nan
5678,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,8953458,2022 Mar 20,2022,35335830
5679,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,8233170,2021 Jun 17,2021,34195015
5680,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,6174339,2018 Aug 28,2018,30344292


In [180]:
# Grouping by Query and Database name so we can place their IDs, Dates, Years, Linked IDs into individual lists. 
summary_df_pmc = (fin_pmc.groupby(['Query_pmc','Db_Name_pmc'])
                .agg({'MainID_List_pmc': lambda x: x.tolist(), "P_Dates_pmc": lambda x: x.tolist(), "P_Years_pmc": lambda x: x.tolist(), "LinkedID_List_pmc": lambda x: x.tolist()})
                .rename({"Query_pmc": "Query", "Db_Name_pmc": "Db_Name", 'MainID_List_pmc': "MainID_List",
                "P_Dates_pmc": "P_Dates", "P_Years_pmc": "P_Years", "LinkedID_List_pmc": "LinkedID_List"}, axis=1)
                .reset_index())

In [181]:
summary_df_pmc

,Query_pmc,Db_Name_pmc,MainID_List,P_Dates,P_Years,LinkedID_List
0,GABA AND GABRA1,pmc,"[6214766, 6543741, 7697095, 6463728, 6180030, ...","[2018 Sep 7, 2019 Jun, 2020 Nov 11, 2019 Feb 6...","[2018, 2019, 2020, 2019, 2018, 2020, 2016, 201...","[30103280, 30571139, 33187258, 30728247, 30305..."
1,GABA AND GABRA2,pmc,"[7697095, 4454465, 6180030, 7698927, 8380214, ...","[2020 Nov 11, 2014 Mar 11, 2018 Oct 10, 2020 N...","[2020, 2014, 2018, 2020, 2021, 2018, 2012, 201...","[33187258, 24631274, 30305619, 33218044, 34417..."
2,GABA AND GABRA3,pmc,"[7697095, 6180030, 7698927, 5538121, 8380214, ...","[2020 Nov 11, 2018 Oct 10, 2020 Nov 18, 2017 J...","[2020, 2018, 2020, 2017, 2021, 2013, 2013, 201...","[33187258, 30305619, 33218044, 28675386, 34417..."
3,GABA AND GABRA4,pmc,"[7697095, 8316187, 6180030, 7698927, 8380214, ...","[2020 Nov 11, 2021 Jun 21, 2018 Oct 10, 2020 N...","[2020, 2021, 2018, 2020, 2021, 2020, 2007, 201...","[33187258, 34152447, 30305619, 33218044, 34417..."
4,GABA AND GABRA5,pmc,"[7697095, 4307650, 6214766, 5972534, 6180030, ...","[2020 Nov 11, 2015 Jan 19, 2018 Sep 7, 1997 Wi...","[2020, 2015, 2018, 1997, 2018, 2020, 2019, 201...","[33187258, 25653499, 30103280, 9017532, 303056..."
5,GABA AND Ivermectin NOT covid-19,pmc,"[5402169, 4645599, 6982461, 7205794, 8240090, ...","[2017 Apr 24, 2015 Jul 30, 2020 Jan 21, 2020 M...","[2017, 2015, 2020, 2020, 2021, 2018, 2020, 202...","[28434401, 26229099, 32021111, 32237037, 34184..."
6,GABA AND Ivermectin NOT covid-19 AND GABRA1,pmc,"[8035410, 3274767, 3949644, 7450100, 6162318, ...","[2021 Apr 9, 2012 Jan 23, 2013 Oct 15, 2020 Ju...","[2021, 2012, 2013, 2020, 2018, 2017, 2017, 201...","[33837214, 22243422, 23941355, 32540960, 30266..."
7,GABA AND Ivermectin NOT covid-19 AND GABRA2,pmc,"[8035410, 4412168, 5650660, 4204805, 5350776, ...","[2021 Apr 9, 2014 Apr 17, 2017 Oct 21, 2013, 2...","[2021, 2014, 2017, 2013, 2013, 2013, 2015]","[33837214, 25931676, 29055038, 23591864, nan, ..."
8,GABA AND Ivermectin NOT covid-19 AND GABRA3,pmc,"[7004153, 5650660, 3892288, 4718212]","[2019 Nov 22, 2017 Oct 21, 2013 Dec 17, 2015 D...","[2019, 2017, 2013, 2015]","[31755996, 29055038, 24528238, 26650440]"
9,GABA AND Ivermectin NOT covid-19 AND GABRA4,pmc,"[5159512, 5650660, 3892288, 4718212, 5659204]","[2016 Nov 7, 2017 Oct 21, 2013 Dec 17, 2015 De...","[2016, 2017, 2013, 2015, 2017]","[27821594, 29055038, 24528238, 26650440, 28215..."


In [182]:
summary_df_pmc = summary_df_pmc.rename(columns={"Query_pmc": "Query", "Db_Name_pmc": "Db_Name"})


In [183]:
# Saving the DataFrame to a CSV file
append_data(summary_df_pmc, "pmcids_OrderPres_NoPubMedDupes.csv", True)

#### Now, we can get the top-3 results per query from the PMC database!

In [ ]:
df_cnt = cited_cnt_table(summary_df_pmc, db_name="pmc")
append_data(df_cnt, "pmc_citation_cnts.csv", True)

In [ ]:
df_citations = pd.read_csv("pmc_citation_cnts.csv")
df_citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043 entries, 0 to 1042
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Query          1043 non-null   object
 1   Db_Name        1043 non-null   object
 2   Id_List        1043 non-null   int64 
 3   Citation_Cnts  1043 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 32.7+ KB


In [ ]:
df_top_3 = get_top_k(df_citations, 3)
df_top_3

,Query,Top_3_Ids,Citation_Cnts
0,GABA AND GABRA1,"[6180030, 5195897, 3412149]","[41, 62, 245]"
1,GABA AND GABRA2,"[6180030, 6870297, 3181829]","[41, 34, 48]"
2,GABA AND GABRA3,"[6180030, 5545734, 3412149]","[41, 29, 245]"
3,GABA AND GABRA4,"[6180030, 4477717, 5301472]","[41, 154, 29]"
4,GABA AND GABRA5,"[4307650, 6180030, 2075237]","[45, 41, 142]"
5,GABA AND Ivermectin NOT covid-19,"[5402169, 6220708, 7008306]","[35, 90, 30]"
6,GABA AND Ivermectin NOT covid-19 AND GABRA1,"[3274767, 6853794, 5650660]","[60, 70, 123]"
7,GABA AND Ivermectin NOT covid-19 AND GABRA2,"[5650660, 4204805, 3892288]","[123, 83, 105]"
8,GABA AND Ivermectin NOT covid-19 AND GABRA3,"[5650660, 3892288, 4718212]","[123, 105, 104]"
9,GABA AND Ivermectin NOT covid-19 AND GABRA4,"[5650660, 3892288, 4718212]","[123, 105, 104]"


In [ ]:
append_data(df_top_3, "pmc_top3_25cited.csv", True)

---

In [92]:
# These rows are all the PMCIDs that do not have corresponding PMIDs. That is why LinkedID_List = 'nan'
pmc_explode[pmc_explode["LinkedID_List"] == 'nan']

,Query,Db_Name,Query_Count,MainID_List,P_Dates,P_Years,LinkedID_List
366,GABA AND GABRA1,pmc,1214,5719066,2017 Nov 30,2017,nan
530,GABA AND GABRA1,pmc,1214,5149585,2016 Dec 1,2016,nan
561,GABA AND GABRA1,pmc,1214,5350815,2014 Jun 10,2014,nan
730,GABA AND GABRA1,pmc,1214,6778823,2019 Jul 3,2019,nan
828,GABA AND GABRA1,pmc,1214,5475995,2017 Mar 20,2017,nan
...,...,...,...,...,...,...,...
5423,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,4272368,2014 Dec 3,2014,nan
5427,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,4049878,2010 Jul 1,2010,nan
5428,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,2908743,2010 Jun 17,2010,nan
5438,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,113,4040166,2013 Nov 23,2013,nan


In [96]:
pmc_explode[pmc_explode["LinkedID_List"] == 'nan'][["Query","Db_Name","MainID_List","P_Dates", "P_Years"]]

,Query,Db_Name,MainID_List,P_Dates,P_Years
366,GABA AND GABRA1,pmc,5719066,2017 Nov 30,2017
530,GABA AND GABRA1,pmc,5149585,2016 Dec 1,2016
561,GABA AND GABRA1,pmc,5350815,2014 Jun 10,2014
730,GABA AND GABRA1,pmc,6778823,2019 Jul 3,2019
828,GABA AND GABRA1,pmc,5475995,2017 Mar 20,2017
...,...,...,...,...,...
5423,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,4272368,2014 Dec 3,2014
5427,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,4049878,2010 Jul 1,2010
5428,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,2908743,2010 Jun 17,2010
5438,GABA AND Ivermectin NOT covid-19 AND Zebrafish,pmc,4040166,2013 Nov 23,2013
